In [1]:
from surgeon_recording.reader import Reader
from glob import glob
from os.path import join
import os
import numpy as np
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
from copy import deepcopy
import pandas as pd

import cv2
import csv
from natsort import natsorted
from surgeon_recording.synchronization import Synchro

from moviepy.editor import *
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

IndentationError: unindent does not match any outer indentation level (synchronization.py, line 359)

In [2]:
synchro = Synchro()

# Manual entries

In [3]:
recording_session = 'SFITS_28_03_22'
subject = 'MAXIME'
camera_ref = "MICROSCOPE"

#synchro mouvement frames
rs_frame = 694
micro_frame = 365
gopro_frame = 1217

#rs_abs_time = 1648459085159.2    #REMPLACER AVEC LES INFOS DES PNG
#rs_nb_frame = 66934

fps_gopro = 59.940
fps_rs = 29.98
fps_micro = 30

frame = {
    "REALSENSE": rs_frame,
    "GOPRO": gopro_frame,
    "MICROSCOPE": micro_frame
}


fps = {
    "GOPRO": fps_gopro,
    "MICROSCOPE": fps_micro,
    "REALSENSE": fps_rs
}

cameras = ['GOPRO', 'MICROSCOPE', 'REALSENSE']
folder = ['complete','segments']


path_to_data_folder = '/home/anaelle/git/surgeon_recording/data'

data_folder = join('..', 'data', recording_session , subject)

In [4]:
print('WARNING: need to create the appropriate folders: GOPRO, MICROSCOPE, REALSENSE, CSV, PNG, complete, segments')

# Convert video

## bag to png to mp4

In [ ]:
# extract the png with the converter (from shell) to the folder PNG in the folder REALSENSE in the SUBJECT folder

camera = 'REALSENSE'
synchro.bag_to_png(path_to_data_folder, data_folder, camera)

In [ ]:
# put the png into MP4 video with given fps
synchro.png_to_MP4(fps_rs, data_folder, camera)

## get segments into one video (gopro, microscope)


In [ ]:
for camera in cameras:
    if camera != 'REALSENSE':
        synchro.concatenate_videos(fps[camera], data_folder, camera)

# Write CSV with absolute time ref

In [5]:
###################################get absolute time from png names + nb of frames of realsense ###################################################
camera = 'REALSENSE'
synchro.get_abs_time_png_names(data_folder, camera, rs_frame)

abs_time_vector = synchro.abs_time_vector[camera]
rs_abs_time = synchro.rs_abs_time 

#####################################get nb of frames for micro and gopro #########################################################################
synchro.get_nb_frame(data_folder, cameras, folder)

for camera in cameras:
    print('Nb of frames of the output' + camera + ' video', synchro.nb_frames[camera])
    
    
########################################### write CSV with corresponding abs time for other cameras ################################################################################
synchro.write_csv_file_camera(data_folder, cameras, folder, frame, fps  )

Total nb of frames of segments for  GOPRO 98287
Total nb of frames of segments for  MICROSCOPE 97236
Total nb of frames of segments for  REALSENSE {}
Nb of frames of the outputGOPRO video 98289
Nb of frames of the outputMICROSCOPE video 48610
Nb of frames of the outputREALSENSE video 43380
[['output_gopro.MP4', 'output_gopro_original.mp4', 'outputGOPRO.mp4']]
[['GP025852.MP4', 'GP015852.MP4', 'GOPR5852.MP4']]
[['output_microscope.MP4', 'outputMICROSCOPE.mp4', 'output_microscope.mp4']]
[['SegmentSDI10013.mp4', 'SegmentSDI10018.mp4', 'SegmentSDI10019.mp4', 'SegmentSDI10014.mp4', 'SegmentSDI10016.mp4', 'SegmentSDI10012.mp4', 'SegmentSDI10015.mp4', 'SegmentSDI10017.mp4']]
[['my_video2.mp4']]
[]


# Segmentation

## read csv data created to store them

In [6]:
synchro.get_csv_data_camera( data_folder, cameras)

#print(synchro.relative_time['MICROSCOPE'])

outputGOPRO.mp4_abs_time.csv
../data/SFITS_28_03_22/MAXIME/GOPRO/CSV/complete/outputGOPRO.mp4_abs_time.csv
outputMICROSCOP_abs_time.csv
../data/SFITS_28_03_22/MAXIME/MICROSCOPE/CSV/complete/outputMICROSCOP_abs_time.csv
REALSENSE_abs_time.csv
../data/SFITS_28_03_22/MAXIME/REALSENSE/CSV/complete/REALSENSE_abs_time.csv


## get closest no of frame and rel time to the ref no frame (on microscope) using absolute time, and export coresp videos (read microscope frame from csv)

In [ ]:

for o in range(len(start_ref_frame_vector)):
    start_ref_frame = start_ref_frame_vector[o]
    stop_ref_frame = stop_ref_frame_vector[o]

    #camera = camera_ref
    start_ref_abs_time = absolute_time[camera_ref][start_ref_frame]
    stop_ref_abs_time = absolute_time[camera_ref][stop_ref_frame]
    #
    pos_start = {}
    pos_stop = {}
    rel_time_start = {}
    rel_time_stop = {}
    abs_time_start = {}
    abs_time_stop = {}
    #for camera in cameras:
    print("start reference abs time:  ", start_ref_abs_time)  
    print("stop reference abs time:  ", stop_ref_abs_time) 
    print("----------------------")
    for camera in cameras:
        pos_start[camera] = []
        pos_stop[camera] = []
        rel_time_start[camera] = {}
        rel_time_stop[camera] = {}
        abs_time_start[camera] = {}
        abs_time_stop[camera] = {}
        if not camera in camera_ref:
            #print(absolute_time[camera])
            [pos_start[camera], abs_time_start[camera]]= synchro.take_closest(absolute_time[camera], start_ref_abs_time, frame[camera])
            print(camera, "start index:       ", pos_start[camera], "abs time:   ", abs_time_start[camera])
            [pos_stop[camera], abs_time_stop[camera]] = synchro.take_closest(absolute_time[camera], stop_ref_abs_time, frame[camera])
            print(camera, "stop index:        ", pos_stop[camera], "abs time:   ", abs_time_stop[camera])
            rel_time_start[camera] = relative_time[camera][pos_start[camera]]
            rel_time_stop[camera] = relative_time[camera][pos_stop[camera]]
            print(camera, "start index:       ", pos_start[camera], "rel time:   ", rel_time_start[camera])
            print(camera, "stop index:       ", pos_stop[camera], "rel time:   ", rel_time_stop[camera])
            print("----------------------")
        else:
            rel_time_start[camera] = relative_time[camera][start_ref_frame]
            rel_time_stop[camera]  = relative_time[camera][stop_ref_frame]
            abs_time_start[camera] = absolute_time[camera][start_ref_frame]
            abs_time_stop[camera]  = absolute_time[camera][stop_ref_frame]
            print(camera, "start index   ", start_ref_frame, "rel time:   ", rel_time_start[camera])
            print(camera, "stop index   ", stop_ref_frame, "rel time:   ", rel_time_stop[camera])
            print("----------------------")
            pos_start[camera] = start_ref_frame
            pos_stop[camera] = stop_ref_frame
            
        #export video from t1 to t2 in seconds


        path_to_data_folder = '/home/anaelle/git/surgeon_recording/data'

        data_folder = join('..', 'data', recording_session , subject)

     
        #for camera in cameras:
        t1 = rel_time_start[camera]
        t2 = rel_time_stop[camera]
    
        #name with abs time ref
        #r1 = abs_time_start[camera]
        #r2 = abs_time_stop[camera]
        #ref = str(r1)[1:7] + "_to_" + str(r2)[1:7] + ".mp4"
    
        #name with no of frame
        r1 = pos_start[camera]
        r2 = pos_stop[camera]
        ref = str(r1) + "_to_" + str(r2) + ".mp4"

        #name with rel time
        #ref = str(t1)[1:7] + "_to_" + str(t2)[1:7] + ".mp4"
    
        #name with rsegment nb
        r1 = index_segment_vector[o]
        ref = "segment_nb_" + str(r1) + ".mp4"
        
        file = [x[2] for x in os.walk(join(data_folder, camera , 'complete'))]
        print(file[0][0])  
        target_name = join(path_to_data_folder, recording_session, subject, camera , 'SEGMENTATION', ref)
        file_name = join(data_folder, camera, 'complete', file[0][0])
        print(target_name)
        ffmpeg_extract_subclip(file_name, t1, t2, targetname=target_name)
    
    #export videos parts according to microscope frames

In [15]:
############# store the frame no of the segments from microscope ref csv file #############################
synchro.get_micro_frame_from_csv(data_folder)


microscope_frames.csv


In [16]:
synchro.export_synchro_videos(path_to_data_folder, data_folder, recording_session, subject, cameras, camera_ref, frame)

start reference abs time:   1648460117596.3809
stop reference abs time:   1648460148629.6384
----------------------
GOPRO stop index:         23585 abs time:    1648460148637.0469
GOPRO start index:        21725 abs time:    1648460117589.3333
GOPRO start index:        21725 rel time:    362.4457791125242
GOPRO stop index:        23585 rel time:    393.4768101435971
----------------------
output_gopro.MP4
/home/anaelle/git/surgeon_recording/data/SFITS_28_03_22/MAXIME/GOPRO/SEGMENTATION/segment_nb_1.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
MICROSCOPE start index    10629 rel time:    354.30000000006623
MICROSCOPE stop index    11560 rel time:    385.33333333342426
----------------------
output_microscope.MP4
/home/anaelle/git/surgeon_recording/data/SFITS_28_03_22/MAXIME/MICROSCOPE/SEGMENTATION/segment_nb_1.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
REALSENSE stop index:         27863 abs time:    1648460148634.5596
REALSENSE 